In [1]:
"""
To Make sure the Best Axon method works and also
to see if can decompose the axon any better 
(especially if can help with crosses and parts that aren't skeletonized             )

"""

'\nTo Make sure the Best Axon method works and also\nto see if can decompose the axon any better \n(especially if can help with crosses)\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [6]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-17 17:56:37,242 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-17 17:56:37,244 - settings - Setting database.user to celiib
INFO - 2021-02-17 17:56:37,244 - settings - Setting database.password to newceliipass
INFO - 2021-02-17 17:56:37,249 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-17 17:56:37,250 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,264 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-17 17:56:37,580 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,613 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-17 17:56:37,962 - settings - Setting enable_python_native_blobs to True


In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu
import datajoint as dj
import neuron_visualizations as nviz
import neuron
import neuron_utils as nru
import proofreading_utils as pru
import classification_utils as clu
import system_utils as su

# Checking the Double Axon Neurons

In [117]:
multi_axon_segs,multi_axon_splits = (minnie.AutoProofreadNeurons2() & "cell_type_predicted='excitatory'"
 & "n_axons>1").fetch("segment_id","split_index")

In [118]:
len(multi_axon_segs)

9346

In [119]:
curr_index = 60
segment_id = multi_axon_segs[curr_index]
split_index = multi_axon_splits[curr_index]
segment_id,split_index

(864691134947394812, 0)

In [109]:
du.plot_proofread_neuron(segment_id,
                        split_index=split_index,
                        plot_errored_faces=True,
                        plot_synapses=True,
                        plot_nuclei=True,
                        plot_axon=True)

Synapsse Results:
Total Number of Synapses = 278
n_valid_syn_centers_presyn = 0
n_errored_syn_centers_presyn = 68
n_valid_syn_centers_postsyn = 200
n_errored_syn_centers_postsyn = 10
Including axon mesh
No axon to plot


In [111]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1


In [112]:
if type(neuron_obj) == tuple:
    test_neuron = neuron_obj[0][split_index]
else:
    test_neuron = neuron_obj

In [113]:
nviz.visualize_neuron(test_neuron,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [107]:
true_axons,true_axon_angles,axon_like_errors = clu.axon_classification(test_neuron,
                        return_axon_angles=True,
                       plot_candidates=False,
                       plot_axons=True,
                        best_axon=True,
                       verbose=True)


Part 1: Axon like branchese 
{'L0': array([ 0,  1,  2,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 18, 20, 21,
       25, 28, 30, 33, 34, 35, 36, 37, 39, 40, 41, 55, 56, 57, 58, 59, 60,
       61, 62]), 'L1': array([ 4,  5,  6,  7, 16, 17, 18, 19, 20])}
Limb L0 soma angle: 62.32 
Limb L1 soma angle: 85.85 
Limb L2 soma angle: 92.93 
Limb L3 soma angle: 119.97 

Part 2: possible_axon_limbs = [1, 2, 3]

Phase 2: Working on Limb L1
nodes_to_eliminate = [ 9 11 12]

Phase 2: Working on Limb L2
nodes_to_eliminate = [0]

Phase 2: Working on Limb L3
nodes_to_eliminate = []

----Working on Limb L1-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L2-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L3-----
No nodes were eliminated so don't need to add back any candidates
Final Candidates

Limb 1
Candidate 0 (orange): array([1, 3, 4, 5, 6, 7, 8])
Candidate 1 (crimson): array([10, 13, 14])
Candidate 2 (pa

In [67]:
# """
# Pseudocode: 
# if there even is an axon
# 1) Get the limb name of the best axon
# 2) Get the concept network of the limb and the starting node
# 3) Get the branches that would be axon
# 4) Delete the branches from the concept network
# 5) For each connected component in the leftover network
# - if not have starting node and all in axon-like: 
# add to list to add to the true axon

# 6) add the new nodes to the axon group
# """

# neuron_obj = test_neuron

# if len(best_limb_branch_dict) > 0:
    
#     #1) Get the limb name of the best axon
#     limb_name_of_axon = list(best_limb_branch_dict.keys())
#     if len(limb_name_of_axon) > 1:
#         raise Excpetion("More than 1 axon key")
    
#     limb_name_of_axon = limb_name_of_axon[0]
    
#     curr_limb = neuron_obj[limb_name_of_axon]
#     #2) Get the concept network of the limb and the starting node
#     conc_net = nx.Graph(curr_limb.concept_network)
#     start_node = curr_limb.current_starting_node
    
#     #3) Get the branches that would be axon
#     axon_branches = best_limb_branch_dict[limb_name_of_axon]
#     axon_like_branches = axon_like_limb_branch_dict[limb_name_of_axon]
    
#     #4) Delete the branches from the concept network
#     conc_net.remove_nodes_from(axon_branches)
    
    
#     #5) For each connected component in the leftover network
#     new_axon_branches = []
#     for conn_comp in nx.connected_components(conc_net):
#         conn_comp = list(conn_comp)
        
#         #- if not have starting node and all in axon-like: 
#         #add to list to add to the true axon
#         axon_like_in_conn_comp = np.intersect1d(axon_like_branches,conn_comp)
#         if start_node not in conn_comp and len(axon_like_in_conn_comp) == len(conn_comp):
#             new_axon_branches += conn_comp
    
    
        
#     if len(new_axon_branches) > 0:
#         best_limb_branch_dict[limb_name_of_axon] = np.array(list(axon_branches) + new_axon_branches)
        
# best_limb_branch_dict

{'L6': array([ 8,  9, 11, 13, 18, 20, 21, 22, 23, 24, 25, 26, 27, 15, 17, 12,  0,
         1,  2,  3,  4,  5,  6,  7])}

# Getting a more high fidelity axon skeleton

In [120]:
segment_id = 864691134947394812
split_index = 0

In [121]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,
                                                      split_index=split_index)

Decompressing Neuron in minimal output mode...please wait
Pulled from Table Decomposition so setting split_index = 0
Number of Neurons found = 1


In [122]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [127]:
neuron_obj

In [129]:
clu.axon_classification(neuron_obj,
                       best_axon=True)


Part 1: Axon like branchese 
{'L3': array([3, 4]), 'L4': array([1]), 'L6': array([0, 1, 2, 3])}
Limb L0 soma angle: 71.03 
Limb L1 soma angle: 66.27 
Limb L2 soma angle: 78.67 
Limb L3 soma angle: 98.66 
Limb L4 soma angle: 98.86 
Limb L5 soma angle: 100.07 
Limb L6 soma angle: 171.82 

Part 2: possible_axon_limbs = [0, 2, 3, 4, 5, 6]

Phase 2: Working on Limb L0
nodes_to_eliminate = []

Phase 2: Working on Limb L2
nodes_to_eliminate = []

Phase 2: Working on Limb L3
nodes_to_eliminate = [0]

Phase 2: Working on Limb L4
nodes_to_eliminate = [0]

Phase 2: Working on Limb L5
nodes_to_eliminate = []

Phase 2: Working on Limb L6
nodes_to_eliminate = [1]

----Working on Limb L0-----
No nodes were eliminated so don't need to add back any candidates

----Working on Limb L2-----
No nodes were eliminated so don't need to add back any candidates

----Working on Limb L3-----
No endpoints were eliminated so don't need to add back any candidates

----Working on Limb L4-----
No endpoints were elimi

({'L6': array([2, 1, 0, 3])}, {'L3': array([4])})

In [130]:
clu.axon_limb_branch_dict(neuron_obj)

{'L6': array([0, 1, 2, 3])}